In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [47]:
faqs="""<To Sherlock Holmes, woh hamesha the woman thi. Maine kabhi nahi suna usko uska koi aur naam lete hue. Uski nazron mein woh apni poori gender ko surpass karti thi aur dominate karti thi. Aisa nahi tha ke usne Irene Adler ke liye koi emotion feel kiya jo love jaisa tha. Sab emotions, aur yeh particular wala, uske cold, precise but admirably balanced mind ko bilkul pasand nahi tha.

Main manta hoon, woh duniya ka sabse perfect reasoning aur observing machine tha, but as a lover, woh apne aap ko ek galat situation mein daal leta. Usne kabhi softer passions ke baare mein baat nahi ki thi, siwaye kisi gibe ya sneer ke. Yeh emotions observers ke liye toh kaafi useful cheez thi—insaan ke motives aur actions ka parda uthane ke liye kaam aati thi. Par ek trained reasoner ke liye inka apne delicate aur finely adjusted temperament mein entry dena matlab ek aise factor ko allow karna tha jo uske mental results par shak daal sakta tha. Jaise ek sensitive instrument mein grit ya uske high-power lenses mein ek crack, waise hi koi strong emotion uski nature ke liye disturb karne waali cheez hoti. Phir bhi, uske liye sirf ek woman thi, aur woh thi Irene Adler, jiska dubious aur questionable memory tha.

Mainne Holmes ko lately zyada nahi dekha tha. Meri shaadi ne hum dono ko door kar diya tha. Meri apni khushi, aur woh home-centered interests jo ek aadmi ko absorb kar lete hain jab usko apna establishment ka master banne ka ehsaas hota hai, ne mera sara attention le liya tha. Wahi Holmes, jo har form of society se nafrat karta tha apne pure Bohemian soul ke saath, Baker Street mein apne purane books ke beech mein chup gaya tha, alternating week to week between cocaine aur ambition, drug ke drowsiness aur apni keen nature ke fierce energy ke beech.

Woh ab bhi, jaise pehle tha, deeply attracted tha crime ke study se, aur apne immense faculties aur extraordinary observation powers ko un clues ko follow karne mein aur mysteries ko solve karne mein lagata jo official police ke liye hopeless ban chuki thi. Time to time, mujhe uske kuch vague kaam ke baare mein sunne ko milta: jaise usko Odessa bulaya gaya tha Trepoff murder ke case mein, ya usne Trincomalee mein Atkinson brothers tragedy ka solution dhoondha tha, ya phir uska mission jo usne Holland ke reigning family ke liye kaafi delicately aur successfully complete kiya tha. In signs ke alawa uski activity ke, jo mujhe sirf daily press ke readers ke saath share hoti thi, mujhe apne purane dost aur companion ke baare mein kuch zyada pata nahi tha.

Ek raat—yeh 20 March, 1888 ki baat hai—main ek patient se wapas aa raha tha (kyun ki ab maine civil practice wapas start kar diya tha), jab meri raah Baker Street ke through guzri. Jaise hi main us well-remembered darwaze ke paas se guza, jo hamesha mere mind mein meri wooing aur Study in Scarlet ke dark incidents ke saath associated rahega, mujhe Holmes se wapas milne ki bohot strong desire hui, aur yeh jaanne ki woh apni extraordinary powers ko kaise use kar raha hai.

Uske rooms bright lights se lit the, aur jaise hi main upar dekha, maine uske tall, spare figure ko do baar dekha, dark silhouette ke against blind ke peeche se guzarte hue. Woh room mein swiftly aur eagerly pace kar raha tha, apna sar apne seene pe jhukaye aur haath apni back ke peeche bandhe hue. Mujhe, jo uske har mood aur habit ko acchi tarah jaanta tha, uske attitude aur manner ne apni kahani bata di. Woh wapas kaam kar raha tha. Woh apne drug-created dreams se uth chuka tha aur kisi naye problem ki talash mein tha. Maine bell bajayi aur mujhe us room mein le jaya gaya jo kabhi mere part ka bhi tha.

Uska manner zyada effusive nahi tha. Kabhi tha bhi nahi; par mujhe lagta hai, woh khush tha mujhe dekhkar. Hardly koi shabd bole bina, but ek kind eye ke saath, usne mujhe ek armchair mein baithne ka indication kiya, apne cigars ka case mere taraf feka, aur corner mein ek spirit case aur gasogene ki taraf ishara kiya. Phir woh fire ke saamne khada ho gaya aur apne singular introspective fashion mein mujhe dekhne laga.

“Shaadi tumhe suit karti hai,” usne kaha. “Mujhe lagta hai, Watson, tumne mere last dekhne ke baad 7 aur aadha pounds badha liya hai.”

“7!” maine jawab diya.

“Indeed, mujhe thoda aur zyada lagta. Bas ek chhoti si extra baat, Watson. Aur tum wapas practice mein ho, main observe kar raha hoon. Tumne mujhe yeh nahi bataya ke tum wapas kaam pe jaa rahe ho.”

“Toh, tum kaise jaane?”

“Maine dekha, maine deduce kiya. Tumne kaafi baar apne aap ko bohot wet kiya hai lately, aur tumhare paas ek bohot clumsy aur careless servant girl hai.>”"""

In [48]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([faqs])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Adding 1 because of zero-padding


In [49]:
# Step 2: Generate input sequences
input_sequences = []
for sentence in faqs.split('\n'):
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [50]:
# Step 3: Pad sequences
max_len = max([len(x) for x in input_sequences])
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

In [51]:
# Step 4: Prepare input and output for the model
X = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [52]:
X.shape

(818, 145)

In [53]:
y.shape

(818, 408)

In [57]:
# Step 5: Define the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Assuming X.shape = (818, 145) and y.shape = (818, 408)
vocab_size = 408  # Number of unique words
input_length = 145  # Length of each input sequence

# Step 1: Define the model with input shape
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=input_length))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))

# Step 2: Build the model explicitly with input shape
model.build(input_shape=(None, input_length))

# Step 3: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Print model summary
model.summary()


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ (None, 145, 100)            │          40,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 145, 150)            │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 150)                 │         180,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 408)                 │          61,608 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 433,608 (1.65 MB)

 Trainable params: 433,608 (1.65 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
# Train the model with early stopping
model.fit(X, y, epochs=200)


Epoch 1/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 580ms/step - accuracy: 0.0425 - loss: 5.5584
Epoch 2/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 21s 583ms/step - accuracy: 0.0511 - loss: 5.5832
Epoch 3/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 21s 625ms/step - accuracy: 0.0610 - loss: 5.4507
Epoch 4/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 19s 585ms/step - accuracy: 0.0462 - loss: 5.5031
Epoch 5/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 582ms/step - accuracy: 0.0403 - loss: 5.3836
Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 21s 602ms/step - accuracy: 0.0500 - loss: 5.1824
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 19s 580ms/step - accuracy: 0.0555 - loss: 5.0656
Epoch 8/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 589ms/step - accuracy: 0.0578 - loss: 4.9683
Epoch 9/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 589ms/step - accuracy: 0.0724 - loss: 4.7498
Epoch 10/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 583ms/step - accuracy: 0.0676 - loss: 4.7302
Epoch 11/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 581ms/step - accuracy: 0.0737 - loss: 4.5247
Epoch 12/200
26/26 ━━━━━━━━━━━

In [63]:
import time
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Take user input for the initial text
text = input("Enter the initial text: ")

# Generate predictions based on user input
for i in range(4):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)


Enter the initial text: woh duniya ka sabse perfect
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
woh duniya ka sabse perfect reasoning
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
woh duniya ka sabse perfect reasoning aur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
woh duniya ka sabse perfect reasoning aur observing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
woh duniya ka sabse perfect reasoning aur observing machine
